<a id = 1></a>
<h1><font color = MidnightBlue>Import Libraries</font></h1>
<hr style="width:100%;height:1.2px;border-width:0;background-color:silver">

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import nltk
import re
import string 

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import keras
from keras.preprocessing import text,sequence
from keras.models import Sequential
from keras.layers import Dense,Embedding,LSTM,Dropout

import warnings
warnings.filterwarnings('ignore')

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



<a id = 2></a>
<h1><font color = MidnightBlue>Load and Check Data</font></h1>
<hr style="width:100%;height:1.2px;border-width:0;background-color:silver">

In [ ]:
real_data = pd.read_csv('/kaggle/input/fake-and-real-news-dataset/True.csv')
fake_data = pd.read_csv('/kaggle/input/fake-and-real-news-dataset/Fake.csv')

In [ ]:
#add column 
real_data['target'] = 1
fake_data['target'] = 0 

In [ ]:
real_data.tail()

In [ ]:
#Merging the 2 datasets
data = pd.concat([real_data, fake_data], ignore_index=True, sort=False)
data.head()

In [ ]:
data.isnull().sum()

<a id = 4></a>
<h1><font color = MidnightBlue>Data Cleaning</font></h1>
<hr style="width:100%;height:1.2px;border-width:0;background-color:silver">

In [ ]:
data['text']= data['subject'] + " " + data['title'] + " " + data['text']
del data['title']
del data['subject']
del data['date']
data.head()

In [ ]:
first_text = data.text[10]
first_text

<a id = 6></a>
<h2><font color = MidnightBlue>Removal of Punctuation Marks and Special Characters</font></h2>

**Let's now remove everything except uppercase / lowercase letters using Regular Expressions.**

In [ ]:
first_text = re.sub('\[[^]]*\]', ' ', first_text)
first_text = re.sub('[^a-zA-Z]',' ',first_text)  # replaces non-alphabets with spaces
first_text = first_text.lower() # Converting from uppercase to lowercase
first_text

<a id = 7></a>
<h2><font color = MidnightBlue>Removal of Stopwords</font></h2>

**Let's remove stopwords like is,a,the... Which do not offer much insight.**

In [ ]:
#nltk.download("stopwords")   
from nltk.corpus import stopwords  

# we can use tokenizer instead of split
first_text = nltk.word_tokenize(first_text)


In [ ]:
first_text = [ word for word in first_text if not word in set(stopwords.words("english"))]
stopwords.words("english")

<a id = 9></a>
<h2><font color = MidnightBlue>Perform it for all the examples</font></h2>
<b>We performed the steps for a single example. Now let's perform it for all the examples in the data.</b>

In [ ]:
#Removal of HTML Contents


#Removal of Punctuation Marks
def remove_punctuations(text):
    return re.sub('\[[^]]*\]', '', text)

# Removal of Special Characters
def remove_characters(text):
    return re.sub("[^a-zA-Z]"," ",text)

#Removal of stopwords 
def remove_stopwords_and_lemmatization(text):
    final_text = []
    text = text.lower()
    text = nltk.word_tokenize(text)
    
    for word in text:
        if word not in set(stopwords.words('english')):
           
            final_text.append(word)
    return " ".join(final_text)

#Total function
def cleaning(text):
    text = remove_punctuations(text)
    text = remove_characters(text)
    text = remove_stopwords_and_lemmatization(text)
    return text

#Apply function on text column
data['text']=data['text'].apply(cleaning)

<a id = 14></a>
<h2><font color = MidnightBlue>Modeling</font></h2>
<hr style="width:100%;height:1.2px;border-width:0;background-color:silver">

<a id = 15></a>
<h2><font color = MidnightBlue>Train Test Split</font></h2>

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data['text'], data['target'], random_state=0)

<a id = 16></a>
<h2><font color = MidnightBlue>Tokenizing</font></h2>

In [ ]:
max_features = 10000
maxlen = 300

In [ ]:
tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(X_train)
tokenized_train = tokenizer.texts_to_sequences(X_train)
X_train = sequence.pad_sequences(tokenized_train, maxlen=maxlen)

In [ ]:
tokenized_test = tokenizer.texts_to_sequences(X_test)
X_test = sequence.pad_sequences(tokenized_test, maxlen=maxlen)

<a id = 17></a>
<h2><font color = MidnightBlue>Training LSTM Model</font></h2>

In [ ]:
batch_size = 256
epochs = 20
embed_size = 100

In [ ]:
model = Sequential()
#Non-trainable embeddidng layer
model.add(Embedding(max_features, output_dim=embed_size, input_length=maxlen, trainable=False))
#LSTM 
model.add(LSTM(units=128 , return_sequences = True , recurrent_dropout = 0.25 , dropout = 0.25))
model.add(LSTM(units=64 , recurrent_dropout = 0.1 , dropout = 0.1))
model.add(Dense(units = 32 , activation = 'relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer=keras.optimizers.Adam(lr = 0.01), loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
tst_real_data = pd.read_csv('/kaggle/input/fake-and-real-news-dataset/True.csv')
#clean

data['text']=data['text'].apply(cleaning)
print(txt)

max_features = 10000
maxlen = 300
tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(txt)
tokenized_train = tokenizer.texts_to_sequences(txt)
X = sequence.pad_sequences(tokenized_train, maxlen=maxlen)

pre=model.predict_classes(X)
pre
(pre < 0.5).astype("int32")

In [ ]:
model.summary()

In [ ]:
history = model.fit(X_train, y_train, validation_split=0.3, epochs=10, batch_size=batch_size, shuffle=True, verbose = 1)

In [ ]:


model.save('LSTM_.h5')

<a id = 18></a>
<h2><font color = MidnightBlue>Analysis After Training </font></h2>

In [ ]:
print("Accuracy of the model on Training Data is - " , model.evaluate(X_train,y_train)[1]*100 , "%")
print("Accuracy of the model on Testing Data is - " , model.evaluate(X_test,y_test)[1]*100 , "%")

In [ ]:
plt.figure()
plt.plot(history.history["accuracy"], label = "Train")
plt.plot(history.history["val_accuracy"], label = "Test")
plt.title("Accuracy")
plt.ylabel("Acc")
plt.xlabel("epochs")
plt.legend()
plt.show()

In [ ]:
plt.figure()
plt.plot(history.history["loss"], label = "Train")
plt.plot(history.history["val_loss"], label = "Test")
plt.title("Loss")
plt.ylabel("Acc")
plt.xlabel("epochs")
plt.legend()
plt.show()

In [ ]:
pred = model.predict_classes(X_test)
print(classification_report(y_test, pred, target_names = ['Fake','Real']))